### 0. To generate state vectors:
#### convert aln to state vectors: \_hmm\_alignment_to_states in anarci.py
##### line 417-425: the modification of the state vector: add the extension of "x" string: need to be modified for C region
##### line 436-446: the actual step to get the item in state vectors: "m":match, "i": insertion, "d":deletion.
#### output: _parse_hmmer_query in anarci.py

### The format of the state vector: 
(state_id, state_type ), si i.e. ((hmm_states[h], state_type),  sequence_index )


### 1. Modify number_imgt function into numbering C region

#### Functions used by the numbering function

In [ ]:
"""
Not sure how the special cases for the C regions are like and whether we need to use this function.
"""
def smooth_insertions(state_vector):
    '''
    The function aims to correct to the expected imgt alignment. Renumbering functions then translate from the imgt scheme to the
    appropriate scheme.        

    Handle insertions made by HMMER that we suspect may be in the wrong position.
    Edge cases include:
        - Insertions at the C terminal of fw1, fw3 and fw3 regions. Can occur when 'conserved' residues have been mutated and the 
          same amino acid appears in the the following CDR (e.g. mutate cysteine at 104 but the CDR3 has one or more cysteines)
        - Same as above possible (but not observed in structure seqs) for N terminal of fw2, fw3 and fw4... TODO
        - Heavily mutated N terminal regions that are partially recognised (e.g. 3gk8 chain H). Insertions should not be allowed 
          before N terminal deletions have been used. Preserve deletion locations that are not N terminal (e.g. 10 in IMGT H) if 
          the gap has been placed by the alignment.

    '''
    # Small overhead doing these corrections but worth it for reducing edge cases.
    
    # Enforce insertion patterns as below. The CDRs are renumbered in each case so that insertions are placed accoring to the scheme    
#  '11111111111111111111111111222222222222333333333333333334444444444555555555555555555555555555555555555555666666666666677777777777'
#  '                        mmmi                         mmmi                                             mmmi                      '
#  '                        mmmi        immm             mmmi      immm                                   mmmi         immm         '

    # Enforce any insertions at the end and beginning of framework regions to be moved into the CDR region for renumbering. 
    enforced_patterns = [ [(25,'m'),(26,'m'),( 27,'m'),( 28,'i')],
                          [(38,'i'),(38,'m'),(39,'m'),(40,'m')],
                          [(54,'m'),(55,'m'),(56,'m'),(57,'i')],
                          [(65,'i'),(65,'m'),(66,'m'),(67,'m')],
                          [(103,'m'),(104,'m'),(105,'m'),(106,'i')],
                          [(117,'i'),(117,'m'),(118,'m'),(119,'m')] ]

    # Insertions in FW1 are only allowed if there are a fewer number of n-terminal deletions made. 

    state_buffer = [] 
    sv = []
    for (state_id, state_type ), si in state_vector:
        if state_id < 23: # Everything before the cysteine at 23.
            state_buffer.append( ((state_id, state_type ), si) )
            reg = -1   
        elif 25 <= state_id < 28: # Add to the buffer 
            state_buffer.append( ((state_id, state_type ), si) )
            reg = 0
        elif 37 < state_id <= 40: # Add to the buffer 
            state_buffer.append( ((state_id, state_type ), si) )
            reg = 1
        elif 54 <= state_id < 57: # Add to the buffer
            state_buffer.append( ((state_id, state_type ), si) )
            reg = 2
        elif 64 < state_id <= 67: # Add to the buffer
            state_buffer.append( ((state_id, state_type ), si) )
            reg = 3
        elif 103 <= state_id < 106: # Add to the buffer
            state_buffer.append( ((state_id, state_type ), si) )
            reg = 4
        elif 116 < state_id <= 119: # Add to the buffer
            state_buffer.append( ((state_id, state_type ), si) )
            reg = 5
        elif len(state_buffer) != 0: # Add the buffer and reset

            # Find the number of insertions in the buffer
            nins = sum( 1 for s in state_buffer if s[0][1] == 'i' ) 

            # If there are insertions, adjust the alignment
            if nins > 0: # We have insertions

                if reg == -1: # FW1, only adjust if there are the same or more N terminal deletions than insertions
                    nt_dels = state_buffer[0][0][0] - 1 # Missing states
                    for (_id, _type ), _si in state_buffer: # Explicit deletion states.
                        if _type == 'd' or _si == None:
                            nt_dels +=1 
                        else: # First residue found
                            break
                    if nt_dels >= nins: # More n terminal deletions than insertions found. Likely misalignment.
                        
                        # Preserve the deleted states structure by using the same match annotations
                        new_states = [ s for s, _ in state_buffer if s[1] == 'm'] 
                        _first = new_states[0][0]

                        # Remove the deletions so that only residue positions are included
                        state_buffer = [ s for s in state_buffer if s[0][1] != 'd' ]

                        # Extend N terminal states backwards from the first match states
                        _add = len( state_buffer ) - len( new_states ) 
                        assert _add >= 0, 'Implementation logic error' # Should be adding a positive number of positions
                        new_states = [ (_,'m') for _ in range( _first - _add, _first ) ] + new_states
                        assert len(new_states)==len(state_buffer), 'Implementation logic error' # Should have the same length

                        # Assign them preserving the order of the sequence. 
                        for i in range( len(state_buffer ) ):
                            sv.append( ( new_states[i], state_buffer[i][1]) )
                    else:
                        sv += state_buffer # The insertions may be incorrect but unknown what to do. Let the alignment place.
                else:
                    # Remove any deletions in the buffer. Unlikely to happen but do anyway
                    state_buffer = [ s for s in state_buffer if s[0][1] != 'd' ]
        
                    # Define the new states defined by the enforced pattern and the length of the buffer
                    if reg % 2: # nterm fw
                        new_states = [enforced_patterns[reg][0]]*max( 0, len(state_buffer)-3) + enforced_patterns[reg][ max( 4-len(state_buffer), 1):]
                    else: # cterm fw
                        new_states = enforced_patterns[reg][:3] + [enforced_patterns[reg][2]]*max( 0, len(state_buffer)-3)
                    # Assign them preserving the order of the sequence. 
                    for i in range( len(state_buffer ) ):
                        sv.append( ( new_states[i], state_buffer[i][1]) )
                                
            else: # Nothing to do - either all match or deletion states.
                sv += state_buffer

            # Add the current state
            sv.append( ((state_id, state_type ), si) )

            # Reset state buffer
            state_buffer = [] 
            
        else: # Simply append 
            sv.append( ((state_id, state_type ), si) )
    

    return sv

In [ ]:
# General function to give annotations for regions that have direct mappings onto the hmm alignment (imgt states)
def _number_regions(sequence, state_vector, state_string , region_string,  region_index_dict, rels, n_regions, exclude_deletions):
    """
    General function to number a sequence and divide it into different regions  
    
    @param sequence: The sequence string
    @param state_vector: The list of states from the aligned hmm
    @param state_string: A string of states for the scheme relative to IMGT (this is X for a direct equivalence, I if needs to be treated as insertion)
    @param region_string: A string of characters that indicate which hmm states are in each regions for this scheme (i.e. how should the sequence be divided up)
    @param region_index_dict: A dictionary converting the characters in region string to an index of the regions. 
    @param rels: The difference of the numbering integer at the *start* of each region
    @param n_regions: The number of regions
    @param exclude_deletions: A list of region indices for which deletion states should not be included. Typically the CDRs. 
                              These will be reannotated in the scheme function. Also allows the reset of insertions. 
    
    @return: A list of lists where each region has been numbered according to the scheme. Some regions will need renumbering. This should be taken care of after the function called.
    
    """

    state_vector = smooth_insertions( state_vector )
    
    _regions = [ [] for _ in range(n_regions) ]
    
    # Initialise the insertion index (-1 is a blank space) and the previous state.
    insertion = -1
    previous_state_id = 1
    previous_state_type = 'd'
    start_index, end_index  = None, None
    
    region = None

    # Iterate over the aligned state vector
    for (state_id, state_type ), si in state_vector:
       
        # Retrieve the region index
        if state_type != "i" or region is None: # BUG_FIX - JD 9/4/15 - do not allow a new region to start as an insertion.
            region = region_index_dict[region_string[state_id-1]] 

       
        # Check the state_types
        if state_type == "m": # It is a match
            
            # Check whether this position is in the scheme as an independent state
            if state_string[state_id-1]=="I": # No, it should be treated as an insertion
                if previous_state_type != 'd': # Unless there was a deletion beforehand in which case this should be a real pos.
                    insertion +=1 # Increment the insertion annotation index
                rels[region] -= 1 # Update the relative numbering from the imgt states
            else: # Yes 
                insertion = -1 # Reset the insertions 
            
            # Add the numbering annotation to the appropriate region list            
            _regions[region].append( ( (state_id + rels[region], alphabet[insertion] ), sequence[si]  ) )
            """
            region should be a value converted from state_id using region_index_dict
            """
            previous_state_id = state_id # Record the previous state ID
            if start_index is None:
                start_index = si
            end_index = si

            previous_state_type = state_type
            
        elif state_type == "i": # It is an insertion
            insertion +=1 # Increment the insertion annotation index
            
            # Add the numbering annotation to the appropriate region list
            _regions[region].append( ( (previous_state_id + rels[region], alphabet[insertion]), sequence[si]  ) )
            if start_index is None:
                start_index = si
            end_index = si

            previous_state_type = state_type

        else: # It is a deletion
            previous_state_type = state_type
            
            # Check whether this position is in the scheme as an independent state
            if state_string[state_id-1]=="I": # No, therefore irrelevant to the scheme.
                rels[region] -= 1 # Update the relative numbering from the imgt states
                continue 
            
            insertion = -1 # Reset the insertions
            previous_state_id = state_id # Record the previous state ID, should not be needed (no delete to insert state transition)


        # Reset the inssertion index if necessary and allowed. (Means the insertion code is meaningless and will be reannotated)
        if insertion >= 25 and region in exclude_deletions:
            insertion = 0 
        
        assert insertion < 25, "Too many insertions for numbering scheme to handle" # We ran out of letters.
            
    return _regions, start_index, end_index



In [1]:
len("87654321|........|....|123|......|..| |........| |.|...|1234567|......|12345677654321|...|......|12|......| |...........| |.........".replace(" ",""))

128

#### The actural numbering function

#### set the initital value of the string to define the region

In [9]:
len("|........|".replace(" ",""))*"5"+len("1234567".replace(" ",""))*"6"+len("|......|".replace(" ",""))*"7"+len("12345677654321".replace(" ",""))*"8"+len("|...|......|".replace(" ",""))*"9"+2*"a"+len("|......|".replace(" ",""))*"b"+len("|...........|".replace(" ",""))*"c"+len("|.........".replace(" ",""))*"d"

'555555555566666667777777788888888888888999999999999aabbbbbbbbcccccccccccccdddddddddd'

In [11]:
len("87654321|........|....|123|......|..| |........| |.|...|1234567|......|12345677654321|...|......|12|......| |...........| |.........".replace(" ",""))

128

In [10]:
len("11111111111111111111111222333333333334444444444555555555566666667777777788888888888888999999999999aabbbbbbbbcccccccccccccdddddddddd")

131

In [ ]:
def number_imgt_C(state_vector, sequence):
    """    
    Apply the IMGT numbering scheme for heavy or light chains
    
    Rules should be implemented using two strings - the state string and the region string. 

    There are 128 states in the HMMs. Treat X as a direct match in IMGT scheme, I is an insertion. (All X's for IMGT)
    XXXXXXXXXXXXXXXXXXXXXXXXXX XXXXXXXXXXXX XXXXXXXXXXXXXXXXX XXXXXXXXXX XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX XXXXXXXXXXXXX XXXXXXXXXXX
    11111111111111111111111111 222222222222 33333333333333333 4444444444 555555555555555555555555555555555555555 6666666666666 77777777777

    Regions - (N.B These do not match up with any particular definition of CDR)
    1. All positions before CDR1
    2. CDR1 positions
    3. Positions between CDR1/2
    4. CDR2 positions
    5. Positions between CDR2/3
    6. CDR positions 105 (inc) to 118 (exc)
    7. Positions after CDR3    
    
    """
    
    # Set up the numbering 
 
    # State string - 'X' means the imgt position exists in the scheme. 'I' means that it should be treated as an insertion of the previous number
    state_string =  'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
    
    # State string for the C region: probably can set the insertion "I" for some points from the beginning
    
    # Region string - regions that should be treated separately in putting the numbering together 
    region_string = '11111111111111111111111111222222222222333333333333333334444444444555555555555555555555555555555555555555666666666666677777777777'
    
    region_string = "11111111111111111111111222333333333334444444444555555555566666667777777788888888888888999999999999aabbbbbbbbcccccccccccccdddddddddd"
    
    """
    region_index_dict = {
                         "1":0,
                         "2":1,
                         "3":2,
                         "4":3,
                         "5":4,
                         "6":5,
                         "7":6
                         }
    This should be changed for the C regions:
    Regions:
    1: A strand
    2: AB turn
    3: B strand
    4: BC loop
    5: C strand
    6: CD turn
    7: D strand
    8: DE turn
    9: E strand
    10: EF turn
    11: F strand
    12: FG loop
    13: G strand
                         
    """
    
    region_index_dict={
        "1":0,
        "2":1,
        "3":2,
        "4":3,
        "5":4,
        "6":5,
        "7":6,
        "8":7,
        "9":8,
        "10":9,
        "11":10,
        "12":11,
        "13":12
    }
    
    # Define how the scheme's numbering differs from IMGT at the start of each region. 
    # This is updated in the loop below
    rels              =  {0:0, 
                          1:0,
                          2:0,
                          3:0,
                          4:0,
                          5:0,
                          6:0,
                          7:0,
                          8:0,
                          9:0,
                          10:0,
                          11:0,
                          12:0,
                          13:0
                          }
    """
    not sure how this value is used and how to modify this. Probably it should be all zeros? (Please double check for this later)
    Their explanation: rels: The difference of the numbering integer at the *start* of each region
    """
    
    
    n_regions = 13

    exclude_deletions = [1,3,5]   
    """ 
    Don't know how this value is used for the C region.
    For V regions:
    exclude_deletions: A list of region indices for which deletion states should not be included. Typically the CDRs. 
                              These will be reannotated in the scheme function. Also allows the reset of insertions. 
    So probably fro the C region, exclude_deletions should be an empty list?
    
    """ 
    
    _regions, startindex, endindex = _number_regions(sequence, state_vector, state_string , region_string,  region_index_dict, rels, n_regions, exclude_deletions)
    
    ###############
    # Renumbering #
    ###############
    
    """
    Probably this part is not needed for the C region?
    Thus, _numbering=_regions
    Please double check for this:
    Not exactly.
    For FG loop, the numbering scheme is similar to CDR3. 
    Thus, a function similar to get_imgt_cdr should be used "symetrically insert the gaps".
    """

    _numbering=_regions[0:11]+[[]]+_regions[12]
    
    """
    Original code for the V region:
    _numbering = [ _regions[0], # Fw1
                   [],          # CDR1
                   _regions[2], # Fw2
                   [],          # CDR2
                   _regions[4], # Fw3
                   [],          # CDR3
                   _regions[6], # Fw4

                 ]
                 
    # The alignment from HMMER should be correct for CDRs 1 and 2. Testing has shown not always the case and 'manual' renumbering 
    # is required as with the other schemes.
    
    # CDR1
    # CDR1 has a range from 27 (inc.) to 39 (exc.) and has a theoretical maximum length of 12.
    cdr1seq    = "".join([ x[1] for x in _regions[1] if x[1] != "-" ])
    cdr1length = len(cdr1seq) 
    si = 0
    prev_state = 26
    for ann in get_imgt_cdr(cdr1length, 12, 27, 39):
        if not ann:
            _numbering[1].append( ((prev_state+1, ' '), '-') )
            prev_state += 1
        else:
            _numbering[1].append( (ann, cdr1seq[si]) )
            prev_state = ann[0]
            si += 1

    # CDR2 
    # CDR2 has a range from 56 (inc.) to 66 (exc.) and has a theoretical length of 10.
    cdr2seq    = "".join([ x[1] for x in _regions[3] if x[1] != "-" ])
    cdr2length = len(cdr2seq)
    si = 0
    prev_state = 55
    for ann in get_imgt_cdr(cdr2length, 10, 56, 66):
        if not ann:
            _numbering[3].append( ((prev_state+1, ' '), '-') )
            prev_state += 1
        else:
            _numbering[3].append( (ann, cdr2seq[si]) )
            prev_state = ann[0]
            si += 1

    # FW3. We allow the HMM to place insertions. Technically all insertion points are taken care of but in reality insertions can
    # and do occur. No specification of where the insertions should be placed.


    # CDR3 
    # CDR3 has a range from 105 (inc.) to 118 (exc.). Insertions are placed on 112 and 111 symetrically. IMGT has a technical 
    # maximum length of 65 (13 positions, 26*2 insertions) . In practice ANARCI will not recognise CDR3s of this length.
    cdr3seq    = "".join([ x[1] for x in _regions[5] if x[1] != "-" ])
    cdr3length = len(cdr3seq)
    if cdr3length > 117: return [], startindex, endindex # Too many insertions. Do not apply numbering. 
    si = 0
    previous_state_id = 104
    for ann in get_imgt_cdr(cdr3length, 13, 105, 118):
        if ann is None:
            _numbering[5].append( ((previous_state_id+1, " "), "-"   ) )
            previous_state_id+=1
        else:
            _numbering[5].append( (ann, cdr3seq[si] ) )
            previous_state_id = ann[0]
            si+=1
    
    
    
    """

    # FG loop (region 12, index:11) 
    # FG loop has a range from 105 (inc.) to 118 (exc.). Insertions are placed on 112 and 111 symetrically. IMGT has a technical 
    # maximum length of 25 (13 positions, 6*2 insertions) . 
    fg_seq    = "".join([ x[1] for x in _regions[11] if x[1] != "-" ])
    fg_length = len(fg_seq)
    if fg_length > 26: return [], startindex, endindex 
    # Too many insertions. Do not apply numbering. Because the longest FG loop is 25 in length, according to IMGT
    """Please double check the value for this: 
    for CDR3, the value is 117, not clear how this value is set, since the longest length for CDR3 should be 91"""
    si = 0
    previous_state_id = 104
    for ann in get_imgt_cdr(fg_length, 13, 105, 118):
        if ann is None:
            _numbering[11].append( ((previous_state_id+1, " "), "-"   ) )
            previous_state_id+=1
        else:
            _numbering[11].append( (ann, fg_seq[si] ) )
            previous_state_id = ann[0]
            si+=1
    
  
    # Return the full vector and the start and end indices of the numbered region of the sequence
    return gap_missing( _numbering ), startindex, endindex

In [ ]:
def get_imgt_cdr(length, maxlength, start, end):
    """
    Symmetrically number a CDR loop (e.g. CDRL1/CDRH2 for IMGT)
    @param length:      Define the length of target CDR
    @param maxlength:   Define the theoretical limit (e.g. L1 = 12 for the IMGT scheme)
    @param start, end:  Start and end position numbers
    """
    annotations = [ None for _ in range(max(length, maxlength)) ]
    if length == 0:
        return annotations
    elif length == 1:
        annotations[0] = (start, ' ')
        return annotations

    front, back = 0, -1
    #az = "ABCDEFGHIJKLMNOPQRSTUVWXYZ" 
    #za = "ZYXWVUTSRQPONMLKJIHGFEDCBA"

    az = alphabet[:-1]
    za = az[::-1]

    for i in range(min(length, maxlength)):
        if i % 2:
            annotations[back] = (end + back, " ")
            back -= 1
        else:
            annotations[front] = (start + front, " ")
            front += 1

    # Add insertions around the centre point
    centrepoint = [ i for i,v in enumerate(annotations) if v == None ]
    if not centrepoint:
        return annotations

    centre_left  = annotations[min(centrepoint)-1][0] # Get the index right before the first None
    centre_right = annotations[max(centrepoint)+1][0] # Get the index right after  the first None

    # For cases with an even max length
    if not maxlength % 2:
        frontfactor, backfactor = maxlength//2, maxlength//2
    # For cases with an odd max length
    else:
        frontfactor, backfactor = (maxlength//2)+1, maxlength//2

    for i in range(max(0, length-maxlength)):
        if not i % 2:
            annotations[back] = (centre_right, za[back + backfactor])
            back -= 1
        else:
            annotations[front] = (centre_left, az[front - frontfactor])
            front += 1
    
    return annotations